In [ ]:
import pandas as pd

In [ ]:
#Loading dataset

In [ ]:
df=pd.read_csv("final_data_in_ML.csv",parse_dates=['Standardized_Date'])
df

In [ ]:
df['DATETIME'] = pd.to_datetime(df['Standardized_Date'].astype(str) + ' ' + df['STANDARDIZED_TIME'].astype(str))
df.dtypes

In [ ]:
df=df[['DATETIME','CLEAR WATER PUMPING FLOW ML','REMARKS']]
df.set_index('DATETIME', inplace=True)
df

In [ ]:
df_daily = df.resample('D').sum(numeric_only=True)
df_daily

In [ ]:
df_daily.plot()

In [ ]:
from prophet import Prophet

In [ ]:
df_final=df_daily.reset_index()[['DATETIME','CLEAR WATER PUMPING FLOW ML']].rename({'DATETIME':'ds','CLEAR WATER PUMPING FLOW ML':'y'},axis='columns')

In [ ]:
df_final

In [ ]:
train = df_final.iloc[:-6]
test = df_final.iloc[-6:]

In [ ]:
train.shape

In [ ]:
test.shape

In [ ]:
test

In [ ]:
m=Prophet()

In [ ]:
m.fit(train)

In [ ]:
future=m.make_future_dataframe(periods=6)
future

In [ ]:
forecast=m.predict(future)
forecast

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from statsmodels.tools.eval_measures import rmse

forecast_test = forecast.iloc[-6:]['yhat'].round(2)
# Calculate errors
error1 = mean_squared_error(test['y'], forecast_test)
error2 = rmse(test['y'], forecast_test)
error3 = mean_absolute_percentage_error(test['y'], forecast_test)
accuracy = (1 - error3) * 100

print(f'MSE Error: {error1:11.10}')
print(f'RMSE Error: {error2:11.10}')
print(f'MAPE Error: {error3:11.10}')
print(f'Accuracy: {accuracy:11.10}')

In [ ]:
forecast[['ds','yhat','yhat_lower','yhat_upper']]

In [ ]:
pd.concat([df_final.set_index('ds')['y'],forecast.set_index('ds')['yhat']],axis=1).plot()

In [ ]:
m.plot(forecast);

In [ ]:
m.plot_components(forecast);

In [ ]:
# Fit the model to the full dataset
model = Prophet()
model.fit(df_final)

# Forecast the next 4 periods
future = model.make_future_dataframe(periods=10)
forecast = model.predict(future)
forecast = forecast[['ds', 'yhat','yhat_lower','yhat_upper']].tail(10).round(2)

# Print summary and forecast
print(forecast)

In [ ]:
pd.concat([df_final.set_index('ds')['y'],forecast.set_index('ds')['yhat']],axis=1).plot()